In [2]:
import pandas as pd
from tqdm.notebook import tqdm

from etnn.tools.training_tools import ConfigStore, load_config
from etnn.routines.run_config import run_config

In [3]:
check_duplicate = True

In [2]:
config = ConfigStore(
    in_dim=15,
    hidden_dim=128,
    out_dim=1,
    k=2,
    dataset=0,
    ds_size=10_000,
    num_gondolas=10,
    num_part_pg=5,
    loss_name='mse',
    optimizer_name='adam',
    num_max_epochs=100, # old: 20
    learning_rate=0.001,
    batch_size=1024,
    early_stop_tol=5,
    use_equal_batcher=False,
    seed=420,
    label_type="tree",  # tree
    final_label_factor=1/1000
)

In [3]:
import time

label_list = ["tree", "default"]
k_list = [1, 2, 3, 4, 5]
hidden_list = [8, 32, 128, 512]
counter = 0
total = len(label_list) * len(k_list) * len(hidden_list)

with tqdm(total=total, desc="Progress", unit="run") as pbar:
    for hidden in hidden_list:
        config.hidden_dim = hidden

        for label in label_list:
            config.label_type = label

            for k in k_list:
                config.k = k

                counter += 1

                run_config(
                    config=config,
                    dataset_path="../datasets",
                    results_folder="../results",
                    verbose=False,
                    check_duplicate=check_duplicate # todo: insert into all others to make clear that this option is being played here
                )

                pbar.update(1)
                pbar.set_postfix({"hidden": hidden, "label": label, "k": k})

Progress:   0%|          | 0/40 [00:00<?, ?run/s]

# Experiments part 2:

In [4]:
tree_intesting_exp = [1,2,3,5,20,24,30]
default_intesting_exp = [0,6,7,25,29,35,36]

In [5]:
# longer experimental runs
with tqdm(total=len(tree_intesting_exp) + len(default_intesting_exp), desc="Progress", unit="run") as pbar:

    for config_idx in tree_intesting_exp + default_intesting_exp:
        # load config
        config = load_config(f"../results/{config_idx}/config.json")

        # make changes to configs
        config.num_max_epochs = 200

        pbar.set_postfix({"running_config_idx": config_idx})

        run_config(
            config=config,
            dataset_path="../datasets",
            results_folder="../results",
            verbose=False
        )

        pbar.update(1)

Progress:   0%|          | 0/14 [00:00<?, ?run/s]

In [6]:
# use batcher
with tqdm(total=len(tree_intesting_exp) + len(default_intesting_exp), desc="Progress", unit="run") as pbar:

    for config_idx in tree_intesting_exp + default_intesting_exp:
        # load config
        config = load_config(f"../results/{config_idx}/config.json")

        # make changes to configs
        config.use_equal_batcher = True

        pbar.set_postfix({"running_config_idx": config_idx})

        run_config(
            config=config,
            dataset_path="../datasets",
            results_folder="../results",
            verbose=False
        )

        pbar.update(1)

Progress:   0%|          | 0/14 [00:00<?, ?run/s]

# New label using C type computation (previous label only S and P)

redo configs for new label for tree

In [4]:
configs = pd.read_csv("../results/config_index.csv")

In [6]:
configs = configs[configs.label_type == "tree"].config_idx
configs

1      1
2      2
3      3
4      4
5      5
10    10
11    11
12    12
13    13
14    14
20    20
21    21
22    22
23    23
24    24
30    30
31    31
32    32
33    33
34    34
40    40
41    41
42    42
43    43
44    44
45    45
46    46
54    54
55    55
56    56
57    57
58    58
59    59
60    60
Name: config_idx, dtype: int64

In [ ]:
for id in tqdm(configs):
    config = load_config(f"../results/{id}/config.json")
    config.label_type = "tree_advanced"

    run_config(
        config=config,
        dataset_path="../datasets",
        results_folder="../results",
        verbose=False
    )

  0%|          | 0/34 [00:00<?, ?it/s]

# Current Progress: here

In [ ]:
models_of_interest = [3,5,24] + [0,25,35]

In [ ]:
datasets_configs = [1, 2]
# change dataset mode
with tqdm(total=len(models_of_interest)*len(datasets_configs), desc="Progress", unit="run") as pbar:

    for config_idx in models_of_interest:
        # load config
        config = load_config(f"../results/{config_idx}/config.json")

        for ds_mode in datasets_configs:
            # make changes to configs
            config.dataset = ds_mode

            pbar.set_postfix({"running_config_idx": config_idx, "ds_mode": ds_mode})

            run_config(
                config=config,
                dataset_path="../datasets",
                results_folder="../results",
                verbose=False
            )

            pbar.update(1)

In [ ]:
loss_configs = ["mse", "mae", "smooth-l1"]
# change loss
with tqdm(total=len(models_of_interest)*len(loss_configs), desc="Progress", unit="run") as pbar:

    for config_idx in models_of_interest:
        # load config
        config = load_config(f"../results/{config_idx}/config.json")

        for loss in loss_configs:
            # make changes to configs
            config.loss_name = loss

            pbar.set_postfix({"running_config_idx": config_idx, "loss": loss})

            run_config(
                config=config,
                dataset_path="../datasets",
                results_folder="../results",
                verbose=False
            )

            pbar.update(1)

In [ ]:
ds_size = [1000, 5000]
# change dataset size
with tqdm(total=len(models_of_interest)*len(ds_size), desc="Progress", unit="run") as pbar:

    for config_idx in models_of_interest:
        # load config
        config = load_config(f"../results/{config_idx}/config.json")

        for ds_s in ds_size:
            # make changes to configs
            config.ds_size = ds_s

            pbar.set_postfix({"running_config_idx": config_idx, "loss": loss})

            run_config(
                config=config,
                dataset_path="../datasets",
                results_folder="../results",
                verbose=False
            )

            pbar.update(1)

In [ ]:
batch_size_list = [64, 128, 256, 512, 1024]
# longer experimental runs
with tqdm(total=len(models_of_interest) * len(batch_size_list), desc="Progress", unit="run") as pbar:
    for config_idx in models_of_interest:
        # load config
        config = load_config(f"../results/{config_idx}/config.json")

        for batch_size in batch_size_list:
            # make changes to configs
            config.batch_size = batch_size

            pbar.set_postfix({"running_config_idx": config_idx, "loss": loss})

            run_config(
                config=config,
                dataset_path="../datasets",
                results_folder="../results",
                verbose=False
            )

            pbar.update(1)